In [11]:
import pandas as pd 
import selenium
import splinter
from bs4 import BeautifulSoup as bs 
from splinter import Browser
import requests
import time
import tweepy
import json
import re
from tweepy import OAuthHandler

import twitter_credentials

def init_browser():
    #starting the browser
    executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
    return Browser('chrome', **executable_path, headless=True)

def close_browser(browser):
    browser.quit()

def scrape_all():
    mars = {}
    mars['news_info'] = scrape_news()
    mars['feat_img'] = f_img()
    mars['twitter'] = twitter()
    mars['data_table'] = table()
    mars['hemisphere_image_urls'] = mars_h_images()
    return  mars
def scrape_news():
    try:
        ## initialize browser and latest_news dict
        browser = init_browser()
        latest_news={}


        ## define url and command browser to visit and parse HTML for posts tag   
        nasa_url= "https://mars.nasa.gov"
        news_url= "https://mars.nasa.gov/news/"
        browser.visit(news_url)
        time.sleep(1)
    
        html = browser.html
        soup = bs(html, "html.parser")
        try:
            #find first news article 
            news = soup.find('div', class_='list_text')

        
            ## collect title, date, description and link of post
            titles = news.find('div', class_='content_title').text
            date = news.find('div', class_='list_date').text
            news_p = news.find('div', class_='article_teaser_body').text
            link = news.a['href']
            ## run it untill the attribute error doesnt show up. 
        except AttributeError:
            print("error")
            
            

        ## add results to latest_news dictionary
        latest_news['news_title'] = titles
        latest_news['news_date'] = date
        latest_news['news_about'] = news_p
        latest_news["news_link"] = f'{nasa_url}{link}'
        return latest_news
    
## close browser    
    finally:        
        close_browser(browser)
def f_img():
    try:
        browser = init_browser()

        url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
        browser.visit(url2)

        #get html code from it 
        html2 = browser.html
        soup2 = bs(html2,'html.parser')

        #get the ending of the url to link the image
        result2 = soup2.find('article', class_ = "carousel_item")
        #identify and reutnr link to listing 
        link = result2.a['data-fancybox-href']
        featured_image_url = f'https://www.jpl.nasa.gov{link}'
        return featured_image_url

    finally:        
        close_browser(browser)

def twitter():
    # set up dict
    mars_weather = {}
    # set up the authication fro the twitter API
    auth = tweepy.OAuthHandler(twitter_credentials.Cons_API_key, twitter_credentials.Cons_API_secret_key )
    auth.set_access_token(twitter_credentials.Access_token, twitter_credentials.Access_token_secret)
    #chose the API of twitter which we will use JSO 
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

    target_user = "MarsWxReport"
    tweet = api.user_timeline(target_user, count =0, page = 0)
    try:
    
        link = ((tweet)[0]['entities']['urls'][0]['url'])
        mars_weather['link'] = link
    except IndexError:
        print('no url')
        
    text = ((tweet)[0]['text'])
    mars_weather['tweet'] = text.replace(link,"")
    
    return mars_weather
def table():
     #read html to get mars facts
    mars_df = pd.read_html("https://space-facts.com/mars/")[0]
    mars_df.columns = ['Description', 'Value']
    mars_df.set_index("Description", inplace = True)
    mars_facts = mars_df.to_html(index = True, header = True, classes="table table-striped")
    return mars_facts
def mars_h_images():
    try:
        browser = init_browser()
        #go to the url to get Mars Hemisphere images 
        url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
        browser.visit(url5)

        #get html code from it 
        html2 = browser.html
        soup5 = bs(html2, "html.parser")
        #find all the listings in the html
        result5 = soup5.find_all('div', class_='item')
         #create an empty list 
        hemisphere_image_urls =[]
        #iterate through the four results and go to their link to get the image url
        for result in result5:
            link = result.a['href']
            browser.visit('https://astrogeology.usgs.gov'+link)
            html6 = browser.html
            soup6 = bs(html6,'html.parser')
            first = soup6.find('div', class_ = 'downloads')
            image_url =first.a['href']
            title = soup6.find('h2', class_ ='title').text
            post = {'title': title, 'img_url': image_url}
            hemisphere_image_urls.append(post)
            browser.back
        return hemisphere_image_urls
    finally:        
        close_browser(browser)

In [12]:
y = scrape_all()

AttributeError: 'dict' object has no attribute 'news_info'

In [10]:

mars_df = pd.read_html("https://space-facts.com/mars/")[0]
mars_df.columns = ['Description', 'Value']
mars_df.set_index("Description", inplace = True)
mars_facts = mars_df.to_html(index = True, header = True, classes="table table-striped")

In [ ]:
table1 = pd.read_html("https://space-facts.com/mars/")[0]

In [ ]:
table2 = pd.DataFrame(table1)

In [ ]:
table2.columns = ['Description', 'Value']

In [ ]:
table2.set_index("Description", inplace = True)

In [ ]:
mars_facts = table2.to_html(index = True, header = True, classes="table table-striped")

In [9]:
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [ ]:
#go to the url nasa mars page to web scrape
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [ ]:
#get html code from the webpage and parse it 
html = browser.html
soup = BeautifulSoup(html,'html.parser')

#get the first news title and first news summary 
result = soup.find('div',  class_ = 'list_text')

news_title = result.find('a', target = '_self').text
news_p = result.find('div', class_='article_teaser_body').text

In [ ]:
if result 

In [ ]:
#go to the url for the featured image 
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [ ]:
#get html code from it 
html2 = browser.html
soup2 = BeautifulSoup(html2,'html.parser')

#get the ending of the url to link to the image 
result2 = soup2.find('article',  class_ = 'carousel_item')
class1 = result2.find('a', class_="button fancybox").text
# Identify and return link to listing
link = result.a['data-fancybox-href']

featured_image_url = 'https://www.jpl.nasa.gov+link'+ link


In [ ]:
# go to the twitter of mars weather 
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)

In [ ]:
html = requests.get('https://twitter.com/marswxreport?lang=en').text
html1 = BeautifulSoup(html,'html.parser')
news1 = html1.find('p',  class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
next_text = news1.text
news5 = news1.find_all("a")
text=("")
text1=("")
for news in news5:
    text1 = news.text
    text = text+text1
mars_weather = next_text.replace(text,'')

In [ ]:
info = scrape_all()

In [ ]:
text    
    

In [ ]:
news

In [ ]:
news = news1.text

In [ ]:
news = news.replace(news5,'')

In [ ]:
news

In [ ]:
#get html code from it 
html3 = browser.html
soup3 = BeautifulSoup(html3,'html.parser')

#get the first tweet on the page 
result3 = soup3.find('div',  class_ = 'js-tweet-text-container')
mars_weather = result3.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text





In [ ]:
# go to the url to get mars facts
url4 = 'https://space-facts.com/mars/'
browser.visit(url4)

In [ ]:
#get html code from it 
html4 = browser.html
soup4 = BeautifulSoup(html4,'html.parser')

#get the first tweet on the page 
result4 = soup4.find('table')
mars_facts = result4.to_html(header = False, index = False)



In [ ]:
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
mars_df.columns=["Description", "Value"]
mars_df.set_index("Description", inplace=True)
mars_df

In [ ]:
# go to the url to get Mars Hemispheres images
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url5)

In [ ]:
#get html code from it 
html5 = browser.html
soup5 = BeautifulSoup(html5,'html.parser')

#find all  the listings in the html 
result5 = soup5.find_all('div', class_="item")


In [ ]:
#Create an empty list 
hemisphere_image_urls = []
# iterate through the four results and go to their link to get the image url
for result in result5:
    link = result.a['href']
    browser.visit('https://astrogeology.usgs.gov'+link)
    html6 = browser.html
    soup6 = BeautifulSoup(html6,'html.parser')
    first = soup6.find('div',class_="downloads")
    image_url = first.a['href']
    title = soup6.find('h2', class_= 'title').text
    post = {"title": title, "img_url": image_url }
    hemisphere_image_urls.append(post)
    
    browser.back()
    
    

In [ ]:
hemisphere_image_urls

In [ ]:
image_url